# SOG Forecast Benchmark Notebook
This notebook demonstrates how to pull sales history from DuckDB and submit it to the Nostradamus forecasting API. Use it as a reference implementation for building repeatable item-level demand forecasts.

## Data Access Utilities
Cell 3 defines SalesHistoryReader, which wraps DuckDB queries for daily and monthly sales history. Update the SQL or connection details here when your storage changes.

In [3]:
import os
import duckdb
from pathlib import Path
from timing_utils import log, start_timer
import pandas as pd


class SalesHistoryReader:
    """
    Utility class for reading sales history from DuckDB.
    """

    def __init__(self, project="SOG", db_path=None):
        self.project = project
        self.db_path = Path(
            db_path or os.getenv("BENCHMARK_DB_PATH", Path.cwd() / "sog.duckdb")
        )

    # ---- DB connection ----
    def get_connection(self):
        return duckdb.connect(str(self.db_path))

    # ---- Queries ----
    def fetch_daily_history(self, item_id):
        sql = """
        SELECT
            item_id,
            sale_date,
            sales
        FROM sales_history
        WHERE project = ? AND item_id = ?
        ORDER BY sale_date
        """
        with self.get_connection() as con:
            return con.execute(sql, [self.project, item_id]).fetchdf()

    def fetch_monthly_history(self, item_id, start_date=None):
        sql = """
        SELECT
            item_id,
            sale_date,
            sales
        FROM sales_history_monthly
        WHERE item_id = ?
        {date_filter}
        ORDER BY sale_date
        """

        date_filter = ""
        params = [item_id]

        if start_date:
            date_filter = "AND sale_date >= ?"
            params.append(start_date)

        sql = sql.format(date_filter=date_filter)

        with self.get_connection() as con:
            return con.execute(sql, params).fetchdf()




## Forecast Orchestration
Cell 4 introduces NostradamusForecaster, responsible for formatting the payload, calling the API, and parsing the response. The helper event_log records each step with UTC timestamps for debugging latency.

In [8]:
import json
import requests
import pandas as pd
from datetime import datetime, timezone


class NostradamusForecaster:
    """
    Formats sales history, calls the Nostradamus API, and parses forecasts.
    """

    def __init__(
        self,
        api_url="https://api.nostradamus-api.com/api/v1/forecast/generate_async",
        forecast_periods=12,
        local_model="auto_arima",
        season_length=12,
        freq="MS",
        mode="local",
    ):
        self.api_url = api_url
        self.forecast_periods = forecast_periods
        self.local_model = local_model
        self.season_length = season_length
        self.freq = freq
        self.mode = mode
        self.event_log = []  # chronological record of forecast steps

    def _record_event(self, action: str) -> None:
        """Append a timestamped action to the event log."""
        self.event_log.append(
            {
                "timestamp": datetime.now(timezone.utc).isoformat().replace("+00:00", "Z"),
                "action": action,
            }
        )

    # -----------------------------
    # History → sim_input_his
    # -----------------------------
    def format_sim_input(self, df_history: pd.DataFrame) -> list:
        self._record_event("format_sim_input_started")
        if df_history.empty:
            self._record_event("format_sim_input_empty_history")
            return []

        df = df_history.copy()
        df["sale_date"] = pd.to_datetime(df["sale_date"]).dt.date
        df = df.sort_values(["item_id", "sale_date"])

        sim_input = [
            {
                "item_id": row["item_id"],
                "actual_sale": float(row["sales"]),
                "day": row["sale_date"].isoformat(),
            }
            for _, row in df.iterrows()
        ]
        self._record_event("format_sim_input_completed")
        return sim_input

    # -----------------------------
    # Build API payload
    # -----------------------------
    def build_payload(self, df_history: pd.DataFrame) -> dict:
        self._record_event("build_payload_started")
        sim_input_his = self.format_sim_input(df_history)

        payload = {
            "sim_input_his": sim_input_his,
            "forecast_periods": self.forecast_periods,
            "mode": self.mode,
            "local_model": self.local_model,
            "season_length": self.season_length,
            "freq": self.freq,
        }
        self._record_event("build_payload_completed")
        return payload

    def build_payload_json(self, df_history: pd.DataFrame) -> str:
        return json.dumps(self.build_payload(df_history), ensure_ascii=False, indent=2)

    # -----------------------------
    # Call API
    # -----------------------------
    def call_api(self, df_history: pd.DataFrame, timeout=30) -> dict:
        self._record_event("call_api_started")
        payload = self.build_payload(df_history)
        self._record_event("call_api_payload_ready")
        r = requests.post(self.api_url, json=payload, timeout=timeout)
        self._record_event("call_api_response_received")
        r.raise_for_status()
        self._record_event("call_api_completed")
        return r.json()

    # -----------------------------
    # Parse API response
    # -----------------------------
    def parse_forecast_df(self, resp: dict) -> pd.DataFrame:
        """
        Returns a dataframe with forecast_date + forecast
        for the first forecast item in the response.
        """
        self._record_event("parse_forecast_df_started")
        if not resp or "forecasts" not in resp or not resp["forecasts"]:
            self._record_event("parse_forecast_df_empty_response")
            return pd.DataFrame(columns=["forecast_date", "forecast"])

        f0 = resp["forecasts"][0]

        df = (
            pd.DataFrame(
                {
                    "forecast_date": pd.to_datetime(f0["forecast_dates"]),
                    "forecast": f0["forecast"],
                }
            )
            .sort_values("forecast_date")
            .reset_index(drop=True)
        )
        self._record_event("parse_forecast_df_completed")
        return df

    # -----------------------------
    # Convenience one-shot method
    # -----------------------------
    def forecast(self, df_history: pd.DataFrame, timeout=30) -> pd.DataFrame:
        self.event_log = []
        self._record_event("forecast_started")
        resp = self.call_api(df_history, timeout=timeout)
        self._record_event("forecast_api_returned")
        forecast_df = self.parse_forecast_df(resp)
        self._record_event("forecast_completed")
        return forecast_df

## Run Example Forecast
Cell 6 configures the item identifier, loads monthly history, and triggers NostradamusForecaster. Adjust ITEM or start_date to experiment with other series.

In [9]:

ITEM = "20-000"
PROJECT = "SOG"

reader = SalesHistoryReader(project=PROJECT)

df_hist = reader.fetch_monthly_history(ITEM, start_date="2022-01-01")

print(df_hist)

forecaster = NostradamusForecaster()
df_fcst = forecaster.forecast(df_hist)

print(df_fcst)

   item_id  sale_date    sales
0   20-000 2022-01-01  1302.00
1   20-000 2022-02-01   468.00
2   20-000 2022-03-01  1221.00
3   20-000 2022-04-01  1998.00
4   20-000 2022-05-01  2950.33
5   20-000 2022-06-01  3328.00
6   20-000 2022-07-01  3152.00
7   20-000 2022-08-01  3591.50
8   20-000 2022-09-01  3440.00
9   20-000 2022-10-01  2946.00
10  20-000 2022-11-01  3369.00
11  20-000 2022-12-01  1268.00
12  20-000 2023-01-01   682.00
13  20-000 2023-02-01  1732.00
14  20-000 2023-03-01  1554.00
15  20-000 2023-04-01  1865.00
16  20-000 2023-05-01  3763.00
17  20-000 2023-06-01  3499.00
18  20-000 2023-07-01  3426.00
19  20-000 2023-08-01  3323.00
20  20-000 2023-09-01  3889.00
21  20-000 2023-10-01  3579.00
22  20-000 2023-11-01  3474.00
23  20-000 2023-12-01  1094.00
24  20-000 2024-01-01   812.00
25  20-000 2024-02-01  1011.00
26  20-000 2024-03-01  1419.00
27  20-000 2024-04-01  2198.00
28  20-000 2024-05-01  3696.00
29  20-000 2024-06-01  3826.00
30  20-000 2024-07-01  3407.00
31  20-0

## Inspect Event Log
Cell 8 exposes forecaster.event_log so you can review timing of each step and pinpoint the slowest stage (usually the API call).

In [6]:
forecaster.event_log

[{'timestamp': '2025-12-26T00:56:35.072288Z', 'action': 'forecast_started'},
 {'timestamp': '2025-12-26T00:56:35.072295Z', 'action': 'call_api_started'},
 {'timestamp': '2025-12-26T00:56:35.072297Z',
  'action': 'build_payload_started'},
 {'timestamp': '2025-12-26T00:56:35.072299Z',
  'action': 'format_sim_input_started'},
 {'timestamp': '2025-12-26T00:56:35.077873Z',
  'action': 'format_sim_input_completed'},
 {'timestamp': '2025-12-26T00:56:35.077879Z',
  'action': 'build_payload_completed'},
 {'timestamp': '2025-12-26T00:56:35.077881Z',
  'action': 'call_api_payload_ready'},
 {'timestamp': '2025-12-26T00:56:37.386016Z',
  'action': 'call_api_response_received'},
 {'timestamp': '2025-12-26T00:56:37.386055Z', 'action': 'call_api_completed'},
 {'timestamp': '2025-12-26T00:56:37.388675Z',
  'action': 'forecast_api_returned'},
 {'timestamp': '2025-12-26T00:56:37.388690Z',
  'action': 'parse_forecast_df_started'},
 {'timestamp': '2025-12-26T00:56:37.396106Z',
  'action': 'parse_forecast_d